Import Packages

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nx
import node2vec
import multiprocessing
import torch
import torch.nn.functional as F
import urllib.request
from torch_geometric.nn import GCNConv
from node2vec import Node2Vec
from urllib.request import urlretrieve

Create ppi human graph

In [5]:
df = pd.read_csv('PP-Pathways_ppi.csv')

edges = list(zip(df['1394'], df['2778']))

G_human = nx.Graph()
G_human.add_edges_from(edges)

print("Human nodes:", len(G_human.nodes()), "edges:", len(G_human.edges()))

Human nodes: 21557 edges: 342352


Retrieve and convert yeast file to graph

In [9]:
from urllib.request import urlretrieve
urllib.request.urlretrieve(
    'http://snap.stanford.edu/deepnetbio-ismb/ipynb/yeast.edgelist',
    'yeast.edgelist'
)
yeast_file = "yeast.edgelist"
G_yeast = nx.read_edgelist(yeast_file)

In [1]:
print('hi')

hi


Generate embeddings for graphs

In [10]:
def generate_embeddings(G, dimensions=32):
    node2vec = Node2Vec(G, dimensions=dimensions, walk_length=10,
                        num_walks=10, workers=8, quiet=True)
    model = node2vec.fit(window=10, min_count=1, batch_words=4)

    # Create DataFrame of embeddings
    emb_df = pd.DataFrame([model.wv.get_vector(str(node)) for node in G.nodes()])
    emb_df["node"] = list(G.nodes())
    return emb_df, model

In [ ]:
emb_yeast, model_yeast = generate_embeddings(G_yeast)
emb_human, model_human = generate_embeddings(G_human)

print("Embeddings (Yeast):", emb_yeast.shape)
print("Embeddings (Human):", emb_human.shape)